https://www.gaussianwaves.com/2010/04/ber-vs-ebn0-for-bpsk-modulation-over-awgn-2/

https://www.mathworks.com/help/comm/ug/bit-error-rate-ber.html

# <font color = blue> <b> Ver referência indicada na página para os cálculos

Para BPSK:

$$
P_s = P_b = Q(\sqrt{\frac{2 Eb}{N0)}})
$$


Para DPSK:

$$
P_s = P_b = 2 Q(\sqrt{\frac{2 Eb}{N0)}}) - 2 Q^2(\sqrt{\frac{2 Eb}{N0)}})
$$

In [1]:
from scipy.special import erfc
import math

In [2]:
def Q(x):
    return (0.5 * erfc(x/(np.sqrt(2))))

In [3]:
Q(0)

0.5

In [4]:
def Pb_BPSK(EbN0):
    return Q(np.sqrt(2*EbN0))

In [5]:
def Pb_DBPSK(EbN0):
    return 2 * Q(np.sqrt(2*EbN0)) - 2 * Q(np.sqrt(2*EbN0))**2

In [6]:
def dB(x):
    return 10 * math.log10(x)

In [7]:
def from_dB(x):
    return math.pow(10, x/10)

In [8]:
Pb_BPSK(from_dB(0))

0.07864960352514258

Aproximadamente o dobro com encoding diferencial

In [9]:
Pb_DBPSK(from_dB(0))

0.1449276867809609

## Para Hard Decision Blocks:


$$
P_b \leq \frac{1}{N} \sum_{m = t + 1}^{N} (m + t) {N \choose m} p^m (1 - p)^{N - m}
$$

In [10]:
from scipy.special import comb

In [11]:
def Pb_Hard_Codes(p, N, t):
    Sum = 0
    for m in range(t+1, N + 1):
        x = (m + t) * comb(N, m, exact = True, repetition = False) * p ** (m) * (1 - p) ** (N - m)
        Sum += x
    return Sum/N

## <font color = green> Valores abaixo corretos com a simulação do GnuRadio.

In [12]:
Pb_BPSK(from_dB(0 - 10 * math.log10(7/4)))

0.14252470370130643

In [13]:
Pb_Hard_Codes(0.07864960352514258, 7, 1)

0.04476969335076263

In [14]:
Pb_Hard_Codes(0.14252470370130643, 7, 1)

0.12338513190559505

## <font color = red> Problema com simulação do diferencial? Por que para os BCHs que corrigem mais de 1 erro o código não está desempenhando bem?

## <font color = Yellow> Provavelmentem a fórmula do Hard Decision Blocks Não Serve para Diff Encoding -> Assume que os erros de bits são aleatórios e não podendo estar juntos

In [15]:
### Para BCH (15, 7, 2) - Valores com cerca de 1% de erro.

In [16]:
Pb_Hard_Codes(Pb_BPSK(from_dB(0 - 10 * math.log10(15/7))), 15, 2)

0.1803196464988525

In [17]:
Pb_BPSK(from_dB(0 - 10 * math.log10(15/7)))

0.16699912791099902

In [18]:
#### Para Diff Encoding

In [19]:
Pb_DBPSK(from_dB(0))

0.1449276867809609

In [20]:
## Diff Encoding Hamming

In [21]:
Pb_DBPSK(from_dB(0 - 10 * math.log10(7/4)))

0.2444228250723225

In [22]:
Pb_Hard_Codes(Pb_DBPSK(from_dB(0 - 10 * math.log10(7/4))), 7, 1)

0.2762370008136536

In [23]:
Pb_DBPSK(from_dB(10 - 0))

7.744186444600035e-06

In [24]:
Pb_Hard_Codes(Pb_DBPSK(from_dB(10 - 10 * math.log10(15/7))), 15, 2)

1.6900642987643003e-06

In [25]:
def Cota_High_Hamming(Ps):
    return (1 - ((1-Ps)**7 + 7 * (1 - Ps)**6 * Ps))

In [26]:
def Cota_High(p, N, t):
    Sum = 0
    for m in range(0, t + 1):
        print(comb(N, m, exact = True, repetition = False))
        x = comb(N, m, exact = True, repetition = False) * p ** (m) * (1 - p) ** (N - m)
        Sum += x
    return 1 - Sum

In [27]:
Cota_High_Hamming(0.01)

0.002031041634940056

In [28]:
Cota_High(0.01, 7, 1)

1
7


0.002031041634940056